In [1]:
# Atlantic salmon RNA-seq counts file - pre-processing
library(matrixStats)
library(parallel)
library(Matrix)

# make 2 columns of SRA_summary file
df = read.delim("chlam_SRA_summary.csv", sep = ',', stringsAsFactors = F)
df["prop_missing"] = 0
df["ranked_corr_in_exp"] = 0
SRA_list = df$SRA

it = 1    
for (SRA_num in SRA_list) {
    
    load(paste0("/data/suresh/chlam/raw_counts/metaExpData", SRA_num, ".Rdata"))
    metaExpData = counts_exp
    
    df$prop_missing[df[,1]==SRA_num] = sum(rowSums(metaExpData) == 0, na.rm = T)     / dim(metaExpData)[1]
    ranked = colRanks(metaExpData, preserveShape = TRUE, ties.method = "average") / dim(metaExpData)[1]
    avgrank = rowMeans(ranked, na.rm = TRUE)        

    # store this to dataframe of gene x SRA
    if (it == 1) {
        avgrank_df = data.frame(rownames(metaExpData), avgrank) 
        colnames(avgrank_df) = c("genes", SRA_num)
        it = 0 
    } else {
        avgrank_df[SRA_num] = avgrank
    }

    df$ranked_corr_in_exp[df[,1]==SRA_num]  = min(cor(ranked, avgrank, use = "pairwise.complete.obs", method = "spearman"), na.rm = TRUE)  

}


write.table(df, file = paste0("chlam_SRA_summary.csv"), sep = ',', row.names = FALSE)
write.table(avgrank_df, file = paste0("Chlamydomonas_reinhardtii_average_rank.csv"), sep = ',', row.names = FALSE)

In [2]:
# add 3rd column to SRA_summary file
df = read.delim(paste0("chlam_SRA_summary.csv"), sep = ',', stringsAsFactors = F)
df["ranked_corr2global"] = 0    
avgrank_df = read.delim(paste0("Chlamydomonas_reinhardtii_average_rank.csv"), sep = ',', stringsAsFactors = F, row.names = 1)

global_avg = rowMeans(avgrank_df, na.rm = T)

for (SRA_num in SRA_list) {
    
    load(paste0("/data/suresh/chlam/raw_counts/metaExpData", SRA_num, ".Rdata"))
    metaExpData = counts_exp

    ranked = colRanks(metaExpData, preserveShape = TRUE, ties.method = "average") / dim(metaExpData)[1]

    temp = cor(ranked, global_avg, use = "pairwise.complete.obs", method = "spearman") 
    temp = temp[order(temp)]
    df$ranked_corr2global[df$SRA==SRA_num] = mean(temp[1:10], na.rm = T) 


    if (df$ranked_corr2global[df$SRA==SRA_num] < .3)  {df$keepInAgg[df$SRA==SRA_num] = -1}
#     if (df$prop_missing[df$SRA==SRA_num] >.5) {df$keepInAgg[df$SRA==SRA_num] = -1}    
}

write.table(df, file = paste0("chlam_SRA_summary.csv"), sep = ',', row.names = FALSE)

In [2]:
buildCoexpNetwork <- function(net, method = "spearman", flag = "rank"){
    
    # Calculate correlation coefficients
    genes = rownames(net)
    net = net[rowSums(net) > 0, ]

    print(paste0("... ",dim(net)[1]," of ",length(genes), " genes have non-zero expression" ))

    net = cor(t(net), method = method)
    
    # Create network
    temp = net[upper.tri(net, diag = T)]
    if (flag == "abs"){
        temp = abs(temp)
    }
    
#     print("...ranking")
    temp = rank(temp, ties.method = "average")  

#     print("...reconstructing matrix")
    net[upper.tri(net, diag = T)] = temp

    net = t(net)
    net[upper.tri(net, diag = T)] = temp

    net = net / max(net, na.rm = T)
    med = median(net, na.rm = T)

    ind = setdiff(genes, rownames(net))   # which genes are missing?

    temp = matrix(med, length(ind), dim(net)[2])
    rownames(temp) = ind

    net = rbind(net, temp)
    temp = matrix(med, dim(net)[1], length(ind))
    colnames(temp) = ind
    net = cbind(net, temp)

    # reorder to original
    net = net[genes, genes]
    diag(net) = 1

    return(net)
}

In [5]:
getHCgenes <- function(SRAlist){
    
    # get high confidence genes
    totals = 0
    
    for (SRA in SRAlist){

        load(paste0("/data/suresh/species/chlam/raw_counts/metaExpData", SRA, ".Rdata"))
        metaExpData = counts_exp        
        A = rowSums(metaExpData >= 10) >= 10
        totals = totals + A
    }
    
    return(names(which(totals >= 20)))
}

In [6]:
df = read.delim("/data/suresh/species/SUMMARY/chlam_SRA_summary.csv", sep = ',', stringsAsFactors = F)
SRA_list = df$SRA[which(df$keepInAgg==1)]
priogenes = getHCgenes(SRA_list)
length(priogenes)
write.csv(priogenes, file = "/data/suresh/species/chlam/PriorityGenes-10count.csv")

[1] 15005

In [8]:
# make coexp matrices
library(utils)

df = read.delim("/data/suresh/species/SUMMARY/chlam_SRA_summary.csv", sep = ',', stringsAsFactors = F)
SRA_list = df$SRA[which(df$keepInAgg==1)]
prioGenes = read.delim("/data/suresh/species/chlam/PriorityGenes-10count.csv", sep = ',', stringsAsFactors = F)
prionet = matrix(0, nrow = 15005, ncol = 15005)

ctr = 0
pb = txtProgressBar(min = 0, max = length(SRA_list), initial = 0)

for (i in SRA_list){
    
    load(paste0("/data/suresh/species/chlam/raw_counts/metaExpData", i, ".Rdata"))
    metaExpData = counts_exp
    ind_row = match(prioGenes[,2], rownames(metaExpData))
    ind_row = ind_row[!is.na(ind_row)]
    
    net = buildCoexpNetwork(metaExpData[ind_row, ])
    
#     filenm = paste('/home/suresh/', i, '_prioNet.Rdata', sep = '')
#     save(net, file = filenm)    
    
    prionet = prionet + net
    
    ctr = ctr + 1
    setTxtProgressBar(pb, ctr) 
    print(paste('Done ', i, sep = ''))    
}

rownames(prionet) = rownames(net)
colnames(prionet) = rownames(net)
save(prionet, file = 'chlam_prioAggNet.Rdata')

[1] "... 14995 of 15005 genes have non-zero expression"
==[1] "Done SRP323000"
[1] "... 14950 of 15005 genes have non-zero expression"
=[1] "Done SRP326188"
[1] "... 14993 of 15005 genes have non-zero expression"
==[1] "Done SRP002284"
[1] "... 14982 of 15005 genes have non-zero expression"
=[1] "Done SRP094014"
[1] "... 14997 of 15005 genes have non-zero expression"
==[1] "Done SRP010062"
[1] "... 15003 of 15005 genes have non-zero expression"
=[1] "Done SRP010563"
[1] "... 15002 of 15005 genes have non-zero expression"
==[1] "Done SRP014795"
[1] "... 14997 of 15005 genes have non-zero expression"
==[1] "Done SRP054846"
[1] "... 15004 of 15005 genes have non-zero expression"
=[1] "Done SRP009466"
[1] "... 15001 of 15005 genes have non-zero expression"
==[1] "Done SRP076181"
[1] "... 15003 of 15005 genes have non-zero expression"
=[1] "Done SRP005483"
[1] "... 15001 of 15005 genes have non-zero expression"
==[1] "Done SRP052618"
[1] "... 15002 of 15005 genes have non-zero expression"
=

In [9]:
prionet[1:4,1:4]

,CHLRE_01g000017v5,CHLRE_01g000033v5,CHLRE_01g000050v5,CHLRE_01g000100v5
CHLRE_01g000017v5,51.00000,30.25204,25.69488,30.56335
CHLRE_01g000033v5,30.25204,51.00000,17.68778,25.40697
CHLRE_01g000050v5,25.69488,17.68778,51.00000,35.78119
CHLRE_01g000100v5,30.56335,25.40697,35.78119,51.00000


In [3]:
# make coexp matrices - already manually removed bad nets from SRA_list
library(utils)

df = read.delim("/data/suresh/species/SUMMARY/chlam_SRA_summary.csv", sep = ',', stringsAsFactors = F)
SRA_list = df$SRA[which(df$keepInAgg==1)]

metanet = matrix(0, nrow = 17742, ncol = 17742)
ctr = 0
pb = txtProgressBar(min = 0, max = length(SRA_list), initial = 0)
for (i in SRA_list){
    
    load(paste0("/data/suresh/species/chlam/raw_counts/metaExpData", i, ".Rdata"))
    metaExpData = counts_exp
    net = buildCoexpNetwork(metaExpData)
    
#     load(paste(i, '_metaNet.Rdata', sep = ''))
#     save(net, file = filenm)
    metanet = metanet + net
    
    ctr = ctr + 1
    setTxtProgressBar(pb, ctr)     
#     print(paste('Done ', i, sep = ''))        
}

rownames(metanet) = rownames(net)
colnames(metanet) = rownames(net)
save(metanet, file = 'chlam_metaAggNet.Rdata')

[1] "... 17155 of 17742 genes have non-zero expression"
==[1] "... 16822 of 17742 genes have non-zero expression"
=[1] "... 17086 of 17742 genes have non-zero expression"
==[1] "... 17129 of 17742 genes have non-zero expression"
=[1] "... 17080 of 17742 genes have non-zero expression"
==[1] "... 17493 of 17742 genes have non-zero expression"
=[1] "... 17400 of 17742 genes have non-zero expression"
==[1] "... 17061 of 17742 genes have non-zero expression"
==[1] "... 17198 of 17742 genes have non-zero expression"
=[1] "... 17360 of 17742 genes have non-zero expression"
==[1] "... 17408 of 17742 genes have non-zero expression"
=[1] "... 17142 of 17742 genes have non-zero expression"
==[1] "... 17365 of 17742 genes have non-zero expression"
=[1] "... 17021 of 17742 genes have non-zero expression"
==[1] "... 16978 of 17742 genes have non-zero expression"
==[1] "... 16557 of 17742 genes have non-zero expression"
=[1] "... 16964 of 17742 genes have non-zero expression"
==[1] "... 17148 of 177

In [ ]:
# save as h5 file

In [4]:
load('chlam_prioAggNet.Rdata')
agg = prionet   # metanet
genes = rownames(agg)

In [5]:
# rank-standardize coexp matrix
temp = agg[upper.tri(agg, diag = T)]
temp = rank(temp, ties.method = "average")  

agg[upper.tri(agg, diag = T)] = temp

agg = t(agg)
agg[upper.tri(agg, diag = T)] = temp

agg = agg / max(agg, na.rm = T)
agg[1:4,1:5]

,CHLRE_01g000017v5,CHLRE_01g000033v5,CHLRE_01g000050v5,CHLRE_01g000100v5,CHLRE_01g000150v5
CHLRE_01g000017v5,1.0000000,0.77907689,0.53492215,0.7925365,0.01737708
CHLRE_01g000033v5,0.7790769,1.00000000,0.09110636,0.5172025,0.12975409
CHLRE_01g000050v5,0.5349222,0.09110636,1.00000000,0.9471775,0.24733681
CHLRE_01g000100v5,0.7925365,0.51720251,0.94717746,1.0000000,0.10538361


In [6]:
library(rhdf5)
hdf5file = 'chlam_prioAggNet.hdf5'                       
suppressWarnings(h5createFile(hdf5file))

h5createDataset(hdf5file, "agg", c(dim(agg)[1],dim(agg)[2]), chunk = c(101,101))
h5createDataset(hdf5file, "row", dims = dim(agg)[1], storage.mode = "character", size = max(1+nchar(genes)))
h5createDataset(hdf5file, "col", dims = dim(agg)[1], storage.mode = "character", size = max(1+nchar(genes)))
h5createDataset(hdf5file, "chunkSize", 1)

h5write(as.matrix(agg), hdf5file, "agg")
h5write(genes, hdf5file, "row")
h5write(genes, hdf5file, "col")
h5write(101, hdf5file, name = "chunkSize")

h5closeAll() 